# demo_2_sentiment.ipynb

_Sentiment classification (rule backend) + hooks for LLM/HF_


## Table of Contents
1. [Setup](#setup)
2. [Imports](#imports)
3. [Data & Quick Demo](#data-demo)
4. [Main Tutorial](#main)
5. [Tips & Troubleshooting](#tips)
6. [Summary](#summary)


## Setup  <a id='setup'></a>

In [ ]:
# %pip -q install spacy geonamescache tqdm
# %pip -q install transformers torch  # optional
# !python -m spacy download en_core_web_sm


## Imports  <a id='imports'></a>

In [ ]:
from spatio_textual.sentiment import SentimentAnalyzer
from spatio_textual.utils import load_spacy_model, Annotator
import pandas as pd


## Data & Quick Demo  <a id='data-demo'></a>

In [ ]:
texts = [
    "I felt safe and relieved when we reached the farmhouse.",
    "We were afraid, hungry, and cold during the march.",
    "They asked us questions.",
]
sa = SentimentAnalyzer("rule")
sa.predict(texts)


## Main Tutorial  <a id='main'></a>
### 1) Annotate + attach sentiment

In [ ]:
nlp = load_spacy_model("en_core_web_sm")
ann = Annotator(nlp)
recs = ann.annotate_texts(texts, file_id="sent_demo", include_text=True)
preds = sa.predict([r["text"] for r in recs])
for r, p in zip(recs, preds):
    r.update({"sentiment_label": p["label"], "sentiment_score": p["score"]})
pd.DataFrame([{k:r.get(k) for k in ["segId","text","sentiment_label","sentiment_score"]} for r in recs])


### 2) Hooking up an HF pipeline (example)

In [ ]:
# from transformers import pipeline
# hf = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
# hf(texts)


## Tips & Troubleshooting  <a id='tips'></a>
- Rule backend is offline and immediate but simplistic; HF/LLM provide richer signals.
- Keep inputs as short segments for better classifier performance.


## Summary  <a id='summary'></a>
You ran sentiment classification with the rule backend and saw how to plug an HF pipeline.
